# DnCNN_V0

Convolutional NN to improve the quality of MRI 
images

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import gc
import datetime
import os

## Data Preprocessing

I will start by loading two of the images in. Then I will select from the originals each only one. Aftwards, I will select the 500 images in good and bad quality from the two image and create the classification label for it.

Due to memory issues I will only select half of the images

In [6]:
X_1 = np.load('/scratch2/ttoebro/data/P10_X.npy')

In [7]:
X_2 = np.load('/scratch2/ttoebro/data/P6_X.npy')

In [8]:
X_3 = np.load('/scratch2/ttoebro/data/P3_X.npy')

In [9]:
X_4 = np.load('/scratch2/ttoebro/data/P1_X.npy')

In [10]:
X = np.concatenate(seq = (X_1, X_2, X_3, X_4), axis=0)

In [11]:
X.shape

(16220, 256, 256)

In [12]:
del X_1, X_2, X_3, X_4

In [13]:
X = X.reshape([X.shape[0], 256, 256, 1])

In [14]:
gc.collect()

7

In [15]:
Y_1 = np.load('/scratch2/ttoebro/data/P10_Y.npy')

In [16]:
Y_2 = np.load('/scratch2/ttoebro/data/P6_Y.npy')

In [17]:
Y_3 = np.load('/scratch2/ttoebro/data/P3_Y.npy')

In [18]:
Y_4 = np.load('/scratch2/ttoebro/data/P1_Y.npy')

In [19]:
Y = np.concatenate(seq = (Y_1, Y_2, Y_3, Y_4), axis=0)

In [20]:
Y.shape

(16220, 256, 256)

In [21]:
del Y_1, Y_2, Y_3, Y_4

In [22]:
Y = Y.reshape([Y.shape[0], 256, 256, 1])

clean up the mess

In [23]:
gc.collect()

12

# Create train and test and validation set

In [24]:
train_frac = 0.8

In [25]:
train_index = int(train_frac * Y.shape[0])

In [26]:
X_train = X[0:train_index,:,:,:]
X_eval = X[train_index:X.shape[0],:,:,:]

In [27]:
Y_train = Y[0:train_index,:,:]
Y_eval = Y[train_index:X.shape[0],:,:,:]

# Create and train CNN

In [ ]:
def cnn_model_fn(features, labels, mode):
    
    # Input Layer
    input_layer = features['x']
    
    # Convolutional layer #1     
    conv1 = tf.layers.conv2d(
        inputs = input_layer,
        filters = 64,
        strides = 2,
        kernel_size = 5,
        padding = "same",
        activation= tf.nn.relu,
        name = "Conv_1")
    
    # Convolutional layer #2
    conv2 = tf.layers.conv2d(
        inputs = conv1,
        filters = 128,
        strides = [2, 2],
        kernel_size = [5, 5],
        padding = "same",
        activation= tf.nn.relu,
        name = "Conv_2")
    
    conv2_bn = tf.layers.batch_normalization(conv2,
                                             name = "Conv_2_bn",
                                             center=True, 
                                             scale=True, 
                                             training=(mode == tf.estimator.ModeKeys.TRAIN))

        
    # Convolutional layer #3
    conv3 = tf.layers.conv2d(
        inputs = conv2_bn,
        filters = 256,
        strides = [2, 2],
        kernel_size = [5, 5],
        padding = "same",
        activation = tf.nn.relu,
        name = "Conv_3")
    
    conv3_bn = tf.layers.batch_normalization(conv3,
                                             name = "Conv_3_bn",
                                             center=True, 
                                             scale=True, 
                                             training=(mode == tf.estimator.ModeKeys.TRAIN))
    
    
    # Deconvolutional layer #1
    deconv1 = tf.layers.conv2d_transpose(
        inputs = conv3_bn,
        filters = 256,
        strides = [2, 2],
        kernel_size = [5, 5],
        padding = "same",
        activation= tf.nn.relu,
        name = "Deconv_1")
    
    deconv1_bn = tf.layers.batch_normalization(deconv1,
                                             name = "deconv_1_bn",
                                             center=True, 
                                             scale=True, 
                                             training=(mode == tf.estimator.ModeKeys.TRAIN))

    
    # Deconvolutional layer #2
    deconv2 = tf.layers.conv2d_transpose(
        inputs = deconv1_bn,
        filters = 128,
        strides = [2, 2],
        kernel_size = [5, 5],
        activation= tf.nn.relu,
        padding = "same",
        name = "Deconv_2")
    
    deconv2_bn = tf.layers.batch_normalization(deconv2,
                                             name = "deconv_2_bn",
                                             center=True, 
                                             scale=True, 
                                            training=(mode == tf.estimator.ModeKeys.TRAIN))

    
    # Deconvolutional layer #3
    deconv3 = tf.layers.conv2d_transpose(
        inputs = deconv2_bn,
        filters = 64,
        strides = [2, 2],
        kernel_size = [5, 5],
        padding = "same", 
        activation= tf.nn.relu,
        name = "Deconv_3")
    
    deconv3_bn = tf.layers.batch_normalization(deconv3,
                                             name = "deconv_3_bn",
                                             center=True, 
                                             scale=True, 
                                             training=(mode == tf.estimator.ModeKeys.TRAIN))
    
    # final covolution to get to 3 layers
    conv4 = tf.layers.conv2d(
        inputs = deconv3_bn,
        filters = 1,
        kernel_size = [1, 1],
        padding = "same",
        activation = tf.nn.relu,
        name = "Conv_4") + input_layer

    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode = mode, predictions=conv4)
    
    # Calculate Loss (for both Train and EVAL modes)
    loss = tf.losses.absolute_difference(labels = labels , predictions = conv4)
    tf.summary.scalar("Value_Loss_Function", loss)

    for var in tf.trainable_variables():
        name = var.name
        name = name.replace(':', '_')
        tf.summary.histogram(name, var)
    merged_summary = tf.summary.merge_all()
        
    # Configure the Training OP (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
        train_op = optimizer.minimize(loss = loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
        
    # Add evaluation metrics
    eval_metric_ops = {
        "accuracy": tf.metrics.mean_absolute_error(
            labels=labels, predictions=conv4)}
    return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)    


Load train and test data

In [ ]:
train_data = X_train
train_labels = Y_train
eval_data = X_eval
eval_labels = Y_eval

Create the estimator

In [ ]:
runconf = tf.estimator.RunConfig(save_summary_steps=5, log_step_count_steps = 10)

In [ ]:
save_dir = "/scratch2/ttoebro/models/" + str(datetime.datetime.now())[0:19].replace("-", "_").replace(" ", "_").replace(":", "_").replace(".", "_")

In [ ]:
## only for this server:
os.system("sudo mkdir -m 777 " + save_dir)

-1

In [ ]:
ImpNet = tf.estimator.Estimator(config=runconf,
    model_fn=cnn_model_fn, model_dir= save_dir)

INFO:tensorflow:Using config: {'_model_dir': '/scratch2/ttoebro/models/2018_11_28_08_37_16', '_tf_random_seed': None, '_save_summary_steps': 5, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 10, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1490e41a20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=10)

logging_hook = tf.train.SummarySaverHook(
     save_steps= 1,
    output_dir="/home/cloud/Training/summary_hook", 
    scaffold=tf.train.Scaffold(summary_op=tf.summary.merge_all())) 

Train the model

In [ ]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=Y_train,
    batch_size=60,
    num_epochs=None,
    shuffle=True)

In [ ]:
ImpNet.train(
    input_fn=train_input_fn,
    steps=10)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /scratch2/ttoebro/models/2018_11_28_08_37_16/model.ckpt.


## Evaluate the model

Make predictions

This is what we train for (our label)

In [ ]:
plt.imshow(Y_eval[1,:,:,0], cmap='gray' )

This is the input of the NN (The noisy image)

In [ ]:
plt.imshow(X_eval[1,:,:,0], cmap='gray' )

This is the true image, can be calculated as the difference of X to the label we train on

In [ ]:
plt.imshow(X_eval[1,:,:,0] - Y_eval[1,:,:,0] , cmap='gray' )

This is an original image. What does our network predict?

In [ ]:
# Evaluate the model and print results
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_eval[1:2,:,:,:]},
    y=X_eval[1:2,:,:,:],
    num_epochs=1,
    shuffle=False)
predict_results = ImpNet.predict(input_fn=predict_input_fn)
predict_results = list(predict_results)

The first dictionary is the image shown above. Although I cannot see much in the picture the network is 99.8% sure that it is original!

In [ ]:
plt.imshow(predict_results[0][:,:,0], cmap='gray')

In [ ]:
plt.imshow(X_eval[1:2,:,:,:].reshape([256, 256]) - predict_results[0][:,:,0], cmap='gray')